In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
import xarray as xr


In [2]:
df = pd.read_csv('../../Data/data_GLDAS/compiled_canada_soil_moisture.csv')

In [3]:
df.columns

Index(['time', 'lat', 'lon', 'SoilMoi0_10cm_inst', 'SoilMoi10_40cm_inst',
       'SoilMoi40_100cm_inst', 'SoilMoi100_200cm_inst'],
      dtype='object')

In [4]:
df = df.drop(['SoilMoi10_40cm_inst', 'SoilMoi40_100cm_inst', 'SoilMoi100_200cm_inst'], axis = 1)
# rename the column 'SoilMoi0_10cm_inst' to 'waterstorage'
df = df.rename(columns={'SoilMoi0_10cm_inst': 'waterstorage'})
# gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df['lon'], df['lat']))
# gdf = gdf.set_crs(epsg=4326, inplace=True)

# gdf.to_parquet('../Data/data_GLDAS/gdf_compiled_canada_soil_moisture.parquet', index=False)

#create a simple df with 4 entries which build a coordinate grid with 0.25° resolution
# df_test = pd.DataFrame(columns=['lon', 'lat', 'waterstorage'])
# lon = []
# lat = []
# for i in range(4):
#     for j in range(4):
#         lon_ = 100.125 + i * 0.25
#         lat_ = 10.125 + j * 0.25
#         lon.append(lon_)
#         lat.append(lat_)
# # list_lon
# df_test = pd.DataFrame({
#     'lon': lon,
#     'lat': lat,
#     'waterstorage': np.random.rand(16),
#     'time': pd.datetime(2023, 1, 1) 
# })

# df_test

In [ ]:


import pandas as pd
import xarray as xr
import numpy as np
import os # Added for output directory creation

# --- Configuration ---
# Assuming df is your loaded DataFrame before this script runs
# Example:
# df = pd.read_csv('your_input_file.csv')

output_dir = '../../output' # Define output directory
output_csv_file = os.path.join(output_dir, 'downsampled_5deg_water_aligned.csv')
output_parquet_file = os.path.join(output_dir, 'downsampled_5deg_water_aligned.parquet')

# Ensure output directory exists
os.makedirs(output_dir, exist_ok=True)


# Define the input grid resolution (used for consistency check, not directly in groupby_bins)
lat_res_in = 0.25
lon_res_in = 0.25
# Define the desired output grid resolution
lat_res_out = 5.0 # Use float for calculations
lon_res_out = 5.0 # Use float for calculations
# --- End Configuration ---

print(f"Starting script...")
print(f"Output CSV: {output_csv_file}")
print(f"Output Parquet: {output_parquet_file}")
print(f"Input resolution: {lat_res_in}° x {lon_res_in}° (assumed)")
print(f"Output resolution: {lat_res_out}° x {lon_res_out}°")

# --- Step 1: Read the CSV data using Pandas ---
# Assume 'df' is already loaded and contains 'time', 'lat', 'lon', 'waterstorage'
print(f"\nUsing pre-loaded DataFrame...")
try:
    # Make sure columns exist before proceeding
    required_cols = ['time', 'lat', 'lon', 'waterstorage']
    if not all(col in df.columns for col in required_cols):
        missing = [col for col in required_cols if col not in df.columns]
        raise ValueError(f"Missing required columns: {missing}")

    # Optional: Convert 'time' column to datetime objects if it's not already
    if not pd.api.types.is_datetime64_any_dtype(df['time']):
        print("Converting 'time' column to datetime...")
        df['time'] = pd.to_datetime(df['time'])

    print("DataFrame ready.")
    print("DataFrame head:")
    print(df.head())

except NameError:
    print("\n---------------------------------------------------------")
    print("Error: DataFrame 'df' not found.")
    print("Please ensure the DataFrame is loaded before running this script.")
    print("---------------------------------------------------------")
    exit()
except ValueError as e:
    print(f"\nError: {e}")
    exit()
except MemoryError: # Keep memory error handling
    print("\n---------------------------------------------------------")
    print("MemoryError: The DataFrame is too large.")
    print("Consider using Dask or chunking if conversion fails later.")
    print("---------------------------------------------------------")
    exit() # Exit the script if memory error occurs
except Exception as e:
     print(f"\nAn unexpected error occurred during DataFrame preparation: {e}")
     exit()


# --- Step 2: Convert Pandas DataFrame to Xarray Dataset ---
print("\nConverting DataFrame to Xarray Dataset...")
try:
    # Check for duplicate indices *before* setting index
    duplicates = df.duplicated(subset=['time', 'lat', 'lon']).sum()
    if duplicates > 0:
        print(f"\nWarning: Found {duplicates} duplicate time/lat/lon combinations. Keeping first occurrence.")
        # Optionally, decide how to handle duplicates (e.g., mean, keep first/last)
        # df = df.groupby(['time', 'lat', 'lon']).mean().reset_index() # Example: average duplicates
        df = df.drop_duplicates(subset=['time', 'lat', 'lon'], keep='first')

    df = df.set_index(['time', 'lat', 'lon'])
    ds = df.to_xarray()

    # Ensure latitude coordinates are sorted (ascending or descending)
    # groupby_bins requires monotonic coordinates
    if not ds.indexes['lat'].is_monotonic_increasing and not ds.indexes['lat'].is_monotonic_decreasing:
         print("Sorting latitude coordinates...")
         ds = ds.sortby('lat')
    if not ds.indexes['lon'].is_monotonic_increasing and not ds.indexes['lon'].is_monotonic_decreasing:
         print("Sorting longitude coordinates...")
         ds = ds.sortby('lon')

    # Optional: Reindex latitude to be decreasing if needed (common standard)
    # if ds['lat'][0] < ds['lat'][-1]:
    #     print("Reindexing latitude to decreasing order...")
    #     ds = ds.reindex(lat=list(reversed(ds['lat'])))

    print("Xarray Dataset created successfully:")
    print(ds)
except KeyError as e:
    print(f"\nError: Column '{e}' not found in DataFrame. Needed for setting index.")
    print("Please ensure your DataFrame has 'time', 'lat', 'lon' columns.")
    exit()
except Exception as e:
    print(f"\nAn error occurred during DataFrame to Xarray conversion: {e}")
    exit()


# --- Step 3: Perform Spatial Aggregation using groupby_bins ---
print(f"\nAggregating data to {lat_res_out}° x {lon_res_out}° grid using groupby_bins...")

try:
    # 3a: Determine the overall extent and create output grid *boundaries*
    min_lat, max_lat = ds['lat'].min().item(), ds['lat'].max().item()
    min_lon, max_lon = ds['lon'].min().item(), ds['lon'].max().item()

    # Calculate the boundaries aligned with the output resolution
    # Use floor for min edge, ceil for max edge, ensuring they align with the grid
    # Add a small epsilon to max extent before ceil to handle points exactly on boundary
    epsilon = 1e-9
    lat_min_bnd = np.floor(min_lat / lat_res_out) * lat_res_out
    lat_max_bnd = np.ceil((max_lat + epsilon) / lat_res_out) * lat_res_out
    lon_min_bnd = np.floor(min_lon / lon_res_out) * lon_res_out
    lon_max_bnd = np.ceil((max_lon + epsilon) / lon_res_out) * lon_res_out

    # Create the bin edges for grouping
    # Use arange up to max_bnd + resolution/2 to ensure the last bin edge is included
    lat_bins = np.arange(lat_min_bnd, lat_max_bnd + lat_res_out / 2, lat_res_out)
    lon_bins = np.arange(lon_min_bnd, lon_max_bnd + lon_res_out / 2, lon_res_out)

    # 3b: Calculate the desired output grid *centroids* (labels for the bins)
    # These will become the new coordinates in the aggregated dataset
    lat_centroids = lat_bins[:-1] + lat_res_out / 2.0
    lon_centroids = lon_bins[:-1] + lon_res_out / 2.0

    print(f"Latitude bins: {lat_bins}")
    print(f"Latitude centroids: {lat_centroids}")
    print(f"Longitude bins: {lon_bins}")
    print(f"Longitude centroids: {lon_centroids}")

    # 3c: Perform the aggregation
    # Group by latitude bins first, calculate the mean for each bin,
    # then group the result by longitude bins and calculate the mean.
    # Use the calculated centroids as the coordinate labels for the resulting bins.
    # 'right=False' means bins are [left, right), matching np.floor logic.
    ds_agg = ds.groupby_bins('lat', bins=lat_bins, labels=lat_centroids, right=False).mean()
    ds_agg = ds_agg.groupby_bins('lon', bins=lon_bins, labels=lon_centroids, right=False).mean()

    # Rename the coordinate dimensions from 'lat_bins'/'lon_bins' back to 'lat'/'lon'
    ds_agg = ds_agg.rename({'lat_bins': 'lat', 'lon_bins': 'lon'})

    print("Aggregation complete.")
    print("Aggregated Dataset:")
    print(ds_agg)

except Exception as e:
    print(f"\nAn error occurred during aggregation with groupby_bins: {e}")
    print("Check if latitude/longitude coordinates are monotonic (sorted).")
    exit()


# --- Step 4: Convert back to DataFrame and save ---
print("\nConverting aggregated Dataset back to DataFrame...")
# Use .reset_index() to turn the coordinates ('time', 'lat', 'lon')
# back into columns for the CSV/Parquet output.
df_agg = ds_agg.to_dataframe().reset_index()

# Optional: Drop rows where the aggregated value is NaN
# This happens if an output grid cell contained no input data points.
nan_count_before = df_agg['waterstorage'].isnull().sum()
if nan_count_before > 0:
    print(f"Dropping {nan_count_before} rows with NaN 'waterstorage' values (empty grid cells).")
    df_agg = df_agg.dropna(subset=['waterstorage'])

print(f"\nSaving aggregated data...")
try:
    # Save to CSV
    print(f"Saving to CSV: {output_csv_file}")
    df_agg.to_csv(output_csv_file, index=False, float_format='%.5f') # Control float precision

    # Save to Parquet (often more efficient for large data)
    print(f"Saving to Parquet: {output_parquet_file}")
    df_agg.to_parquet(output_parquet_file, index=False)

    print("Aggregated data saved successfully.")
    print("\nFinal DataFrame snippet:")
    print(df_agg.head())

except Exception as e:
    print(f"\nAn error occurred while saving the output files: {e}")

print("\nScript finished.")